## FWI Metrics Data Store Organization

### Top level
```text
/<data version>
    /<netcdf or cog>
```

### File level
**NetCDF `/v0/netcdf`** 
```text
/fwi
    /<ensemble or model member>
        /<ensemble stat>
            /<cadence>
                /<ssp|historical>
                    /<yyyy-mm | yyyy>
                        /<ensemble stat>_<ssp>_fwi_metrics_<cadence>_<yyyy-mm>_<fwi metric>.tif
###
# Sample/Src
src_url = s3://cmip6-staging/Sample/FWI/Yearly/MME/MME50_ssp245_fwi_metrics_yearly_2100.nc
# Dst
dst_url = s3://veda-nex-gddp-cmip6-public/v0/netcdf/fwi/mme/mme50/yearly/ssp245/2100/mme50_ssp245_fwi_metrics_yearly_2100.nc
###
```

In [1]:
import cmip6_file_organization

In [2]:
import boto3
# # if running locally, uncomment to set user role for session
# boto3.setup_default_session(profile_name="deltawest")
client = boto3.client("s3")

In [3]:
STAGING_BUCKET = "cmip6-staging"
DST_BUCKET = "veda-nex-gddp-cmip6-public"

EXPERIMENTS = ["historical", "ssp245"]

## Publish all yearly netcdfs to public data store

In [4]:
error_responses = []
verbose = False
dryrun = True

for experiment in EXPERIMENTS:
    prefix = f"Sample/FWI/Yearly/MME/MME50_{experiment}_"
    r = client.list_objects_v2(
        Bucket = STAGING_BUCKET,
        Prefix = prefix,
    )
    if verbose or dryrun:
        print(f"\n{r['KeyCount']} objects for {prefix=} in {STAGING_BUCKET=}")
    objects = r["Contents"]
    
    for obj in objects:
        
        src_key = obj["Key"]
        nc_key = cmip6_file_organization.generate_yearly_fwi_metrics_key(src_key, "v0", pub_type="netcdf")            
        
        # Skip existing      
        if (client.list_objects_v2(
            Bucket = DST_BUCKET,
            Prefix = nc_key,
        ))["KeyCount"] > 0:
            if verbose or dryrun:
                print(f"CONTINUE {nc_key=} already uploaded to {DST_BUCKET}")
            continue
            
        if dryrun:
            print(f"WOULD HAVE uploaded {nc_key=} to {DST_BUCKET=}")
            continue
            
        if verbose:
            print(f"START to upload {nc_key=} to {DST_BUCKET=}")

        # upload and confirm success
        r = client.copy_object(Bucket=DST_BUCKET, CopySource=f"{STAGING_BUCKET}/{src_key}", Key=nc_key)
        if r["ResponseMetadata"]["HTTPStatusCode"] != 200:
            print(f"WARNING unable to upload {nc_key=}")
            error_responses.append(r)
            
print(f"\nCOMPLETED with {len(error_responses)} errors")
if error_responses:
    print(error_responses)


65 objects for prefix='Sample/FWI/Yearly/MME/MME50_historical_' in STAGING_BUCKET='cmip6-staging'
CONTINUE nc_key='v0/netcdf/fwi/mme/mme50/yearly/historical/1950/mme50_historical_fwi_metrics_yearly_1950.nc' already uploaded to veda-nex-gddp-cmip6-public
CONTINUE nc_key='v0/netcdf/fwi/mme/mme50/yearly/historical/1951/mme50_historical_fwi_metrics_yearly_1951.nc' already uploaded to veda-nex-gddp-cmip6-public
CONTINUE nc_key='v0/netcdf/fwi/mme/mme50/yearly/historical/1952/mme50_historical_fwi_metrics_yearly_1952.nc' already uploaded to veda-nex-gddp-cmip6-public
CONTINUE nc_key='v0/netcdf/fwi/mme/mme50/yearly/historical/1953/mme50_historical_fwi_metrics_yearly_1953.nc' already uploaded to veda-nex-gddp-cmip6-public
CONTINUE nc_key='v0/netcdf/fwi/mme/mme50/yearly/historical/1954/mme50_historical_fwi_metrics_yearly_1954.nc' already uploaded to veda-nex-gddp-cmip6-public
CONTINUE nc_key='v0/netcdf/fwi/mme/mme50/yearly/historical/1955/mme50_historical_fwi_metrics_yearly_1955.nc' already uplo